In [10]:
!pip install watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.8 MB/s eta 0:00:00


In [11]:
%load_ext watermark

In [12]:
%watermark -a "Rishit Kapoor 21BAI1046" -u -d -v -m

Author: Rishit Kapoor 21BAI1046

Last updated: 2024-02-02

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.58+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'fruits:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5857%2F2609027%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240202%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240202T165424Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D245f100ae82b7a4e434168e560cbf3a68140f3c639188c6285b06515372cead86a843cf3452960f07874fe8fe3faec9b2ad3edac600ac2d51111f5b3bbed6a1e48be1277c68ddfb6741b605a164608c1dba0ef226b64b5fc69801a6f40cd9d9200b92dc6d1115489bc4f10145fbabb8de2b68bce7d313204c7a17792941febb47aaf062717e90300d0b23c984e612f11f58493c78dcd226f8764527efe2a7fc7924a0fc9f385891f7c7eb5f8b93d1ef5d2ebc640bfcf95f4151861c8251231ed5c37f93d450997325bda2d4ad1ee7648ac6cfeec5e71391e7b110d0e8aa0a6b413494a90d6e6b0f0a287f88c786861d749a03da042638d6efab857da600a1a90'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1379678841 bytes downloaded
Downloaded and uncompressed: fruits
Data source import complete.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (20,8)

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [4]:
img_size = 100
batch_size = 120

train_datagen = ImageDataGenerator(rescale=1/255.,rotation_range=15,width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,horizontal_flip=True,fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory('/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training',target_size=(img_size, img_size),batch_size=batch_size,shuffle=True,class_mode='sparse')
test_generator = test_datagen.flow_from_directory('/kaggle/input/fruits/fruits-360_dataset/fruits-360/Test',target_size=(img_size, img_size),batch_size=batch_size,shuffle=False,class_mode='sparse')

Found 67692 images belonging to 131 classes.
Found 22688 images belonging to 131 classes.


In [5]:
import os

#acquiring a list of all class names

class_names_train = os.listdir('/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training')
print(class_names_train)

['Rambutan', 'Pepper Red', 'Fig', 'Corn Husk', 'Melon Piel de Sapo', 'Pear Red', 'Cherry 2', 'Potato Red', 'Physalis with Husk', 'Grapefruit White', 'Mangostan', 'Apple Golden 2', 'Pepper Yellow', 'Nectarine', 'Walnut', 'Pear', 'Cactus fruit', 'Peach', 'Pineapple', 'Apricot', 'Kumquats', 'Physalis', 'Apple Red 2', 'Tamarillo', 'Avocado', 'Tangelo', 'Redcurrant', 'Huckleberry', 'Tomato Yellow', 'Onion Red', 'Limes', 'Pear Williams', 'Maracuja', 'Mango', 'Onion Red Peeled', 'Dates', 'Grape Pink', 'Pomegranate', 'Papaya', 'Carambula', 'Tomato Maroon', 'Banana', 'Cherry Wax Red', 'Watermelon', 'Pear Forelle', 'Apple Granny Smith', 'Mango Red', 'Raspberry', 'Apple Golden 3', 'Tomato 2', 'Nut Forest', 'Cocos', 'Tomato Heart', 'Apple Pink Lady', 'Onion White', 'Cherry Wax Black', 'Salak', 'Grape White 3', 'Apple Red Delicious', 'Tomato 1', 'Tomato 3', 'Pepino', 'Grape White 2', 'Cherry Rainier', 'Cucumber Ripe 2', 'Potato Red Washed', 'Nut Pecan', 'Tomato Cherry Red', 'Guava', 'Apple Red Yell

In [6]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()

# Fit the LabelEncoder object to the class names to obtain the class labels
le.fit(class_names_train)

# Convert the class names to their corresponding integer labels
class_labels_train = le.transform(class_names_train)

In [7]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout

model = Sequential()

model.add(Conv2D(64, (3,3), input_shape= (img_size,img_size,3), activation = 'relu', padding = 'same')) #padding = same size output
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D())

model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D())

model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(131, activation = 'softmax'))


model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 50, 50, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 25, 25, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 25, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 12, 12, 128)       0

In [ ]:
from keras.callbacks import EarlyStopping

#early_stopping = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=1,
    batch_size = 120
)

  9/565 [..............................] - ETA: 43:51 - loss: 1.5237 - accuracy: 0.5176

In [ ]:
result = model.evaluate(test_generator)